<a href="https://colab.research.google.com/github/mdorodch/ITSC3155-Demos/blob/master/Hour8_endToend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install trl #transformer reinforcement learning


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
# 0. imports
import torch
from transformers import GPT2Tokenizer

from trl import AutoModelForCausalLMWithValueHead
from trl import PPOConfig, PPOTrainer

In [ ]:
# 1. load a pretrained model
model = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
model_ref = AutoModelForCausalLMWithValueHead.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 2. initialize trainer
arguments = {"batch_size": 1}
config = PPOConfig(**arguments)
ppo_trainer = PPOTrainer(config, model, model_ref, tokenizer)

/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_trainer.py:239: UserWarning: No dataset is provided. Make sure to set config.batch_size to the correct value before training.
  warnings.warn(


In [ ]:
# 3. encode a query
query_txt = "This morning I went to the "
query_tensor = tokenizer.encode(query_txt, return_tensors="pt").to(model.pretrained_model.device)

In [ ]:
# 4. generate model response
generation_args = {"min_length": -1,  # don't ignore the EOS token
                    "top_k": 0.0,       # no top-k sampling
                    "top_p": 1.0,       # no nucleus sampling
                    "do_sample": True,
                    "pad_token_id": tokenizer.eos_token_id,# most decoder models don't have a padding token - use EOS token instead
                    "max_new_tokens": 32,}

response_tensor = ppo_trainer.generate([item for item in query_tensor], return_prompt=False, **generation_args)
response_txt = tokenizer.decode(response_tensor[0])
print(response_txt)

vernacular and found myself at a bar, cook, with a wife. Buggas together in the middle of the hall. Their text lists angelic date


In [ ]:
# 5. define a reward for response
# (this could be any reward such as human feedback or output from another model)
reward = [torch.tensor(1.0, device=model.pretrained_model.device)]
print(reward)

[tensor(1., device='cuda:0')]


In [ ]:
# 6. train model with ppo
train_stats = ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)
print(train_stats)


{'objective/kl': 0.0, 'objective/kl_dist': 0.0, 'objective/logprobs': array([[ -8.146127  ,  -2.2490797 ,  -2.6608636 ,  -0.6304363 ,
         -1.6609955 ,  -9.227807  ,  -1.8503934 ,  -0.7273636 ,
         -3.4500875 ,  -2.285384  ,  -2.8818974 ,  -3.0691516 ,
         -1.5126315 ,  -3.39899   ,  -2.631049  , -10.909885  ,
         -2.5386026 ,  -4.6364646 ,  -1.2329607 ,  -5.3302116 ,
         -2.397719  ,  -7.8588104 ,  -8.311342  ,  -5.083457  ,
         -9.240276  ,  -3.2596421 ,  -1.3474137 ,  -2.6704834 ,
         -0.1929232 ,  -0.42045537,  -7.426415  ,  -1.3062041 ,
         -5.853076  ,  -9.540996  ,  -7.971566  , -11.521053  ,
         -0.7778777 ,  -9.440994  ]], dtype=float32), 'objective/ref_logprobs': array([[ -8.146127  ,  -2.2490797 ,  -2.6608636 ,  -0.6304363 ,
         -1.6609955 ,  -9.227807  ,  -1.8503934 ,  -0.7273636 ,
         -3.4500875 ,  -2.285384  ,  -2.8818974 ,  -3.0691516 ,
         -1.5126315 ,  -3.39899   ,  -2.631049  , -10.909885  ,
         -2.538602